In [ ]:
# !pip install numpy
# !pip install pillow
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python
# !pip install matplotlib
# !pip install seaborn

# # using dataset https://www.kaggle.com/datasets/stoicstatic/face-recognition-dataset

In [ ]:

import random
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.applications.mobilenet  import MobileNet
from keras.applications.mobilenet  import preprocess_input


random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)


In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())

In [ ]:
FOLDER_FACE_PATH = './ExtractedFaces'

def readImage(index):
    path = os.path.join(FOLDER_FACE_PATH, index[0], index[1])
    img = cv2.imread(path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def splitDataset(ratio=0.8):
    train = {}
    test = {}
    folders = os.listdir(FOLDER_FACE_PATH)
    folderTrainSize = int(len(folders) * ratio)
    random.shuffle(folders)

    trainFolders = folders[:folderTrainSize]
    testFolders = folders[folderTrainSize:]
    for folder in trainFolders:
        files = os.listdir(os.path.join(FOLDER_FACE_PATH, folder))
        train[folder] = files

    for folder in testFolders:
        files = os.listdir(os.path.join(FOLDER_FACE_PATH, folder))
        test[folder] = files
            
    return train, test
train, test = splitDataset(0.9)


In [ ]:

def createTriplet(train):
    # anchor and positive are from same class, negative is from different class
    triplets = []
    folders = list(train.keys())

    for folder in folders:
        files = train[folder]
        if len(files) < 2:
            continue
        anchorPositivePairs = []
        for i in range(len(files)):
            for j in range(i + 1, len(files)):
                anchorPositivePairs.append((files[i], files[j]))

        for anchor, positive in anchorPositivePairs:
            negativeFolder = folder
            while negativeFolder == folder:
                negativeFolder = random.choice(folders)
            negativeFiles = train[negativeFolder]
            negative = random.choice(negativeFiles)
            triplets.append(((folder, anchor), (folder, positive), (negativeFolder, negative)))
            
    random.shuffle(triplets)
    return triplets

tripletsTrain = createTriplet(train)
tripletsTest = createTriplet(test)
print (len(tripletsTrain), 'triplets created')
print (len(tripletsTest), 'total triplets created')

# Show image triplet
for i in range(3):
    print(tripletsTrain[i])
    anchor = readImage(tripletsTrain[i][0])
    positive = readImage(tripletsTrain[i][1])
    negative = readImage(tripletsTrain[i][2])

    plt.figure(figsize=(10,5))
    plt.subplot(1,3,1)
    plt.imshow(anchor)
    plt.title('Anchor')
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(positive)
    plt.title('Positive')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(negative)
    plt.title('Negative')
    plt.axis('off')

    plt.show()




In [ ]:
def batchGenerator(triplets, batchSize=32):
    while True:
        anchorBatch = []
        positiveBatch = []
        negativeBatch = []
        for i in range(batchSize):
            triplet = random.choice(triplets)
            anchor = readImage(triplet[0])
            positive = readImage(triplet[1])
            negative = readImage(triplet[2])

            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)

            anchorBatch.append(anchor)
            positiveBatch.append(positive)
            negativeBatch.append(negative)

        anchorBatch = tf.convert_to_tensor(np.array(anchorBatch, dtype="float32"))
        positiveBatch = tf.convert_to_tensor(np.array(positiveBatch, dtype="float32"))
        negativeBatch = tf.convert_to_tensor(np.array(negativeBatch, dtype="float32"))
        labels = tf.zeros((batchSize, 1), dtype=tf.float32)

        yield (anchorBatch, positiveBatch, negativeBatch), labels





In [ ]:
def getEmbeddingModel(inputShape):
    baseModel = MobileNet(input_shape=inputShape, include_top=False, weights='imagenet', pooling='avg')
    for layer in baseModel.layers:
        layer.trainable = False

    model = tf.keras.Sequential([
        baseModel,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name='Embedding')
    return model

In [ ]:
#  Create siamese network model with triplet loss

# Distance layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
class DistanceLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)
    

def getSiameseModel(inputShape=(128, 128, 3)):
    embeddingModel = getEmbeddingModel(inputShape)

    anchorInput = tf.keras.layers.Input(name='anchor', shape=inputShape)
    positiveInput = tf.keras.layers.Input(name='positive', shape=inputShape)
    negativeInput = tf.keras.layers.Input(name='negative', shape=inputShape)

    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)

    # Distance layer
    distances = DistanceLayer()(anchorEmbedding, positiveEmbedding, negativeEmbedding)

    model = tf.keras.Model(inputs=[anchorInput, positiveInput, negativeInput], outputs=distances, name='SiameseNetwork')
    return model

SiameseModel  = getSiameseModel()
SiameseModel.summary()

In [ ]:
# Custom training step with triplet loss
class SiameseModelClass(tf.keras.Model):
    def __init__(self, siameseModel, margin=0.5):
        super(SiameseModelClass, self).__init__()
        self.siameseModel = siameseModel
        self.margin = margin
        self.lossTracker = tf.keras.metrics.Mean(name='loss')

    def call(self, inputs):
        return self.siameseModel(inputs)

    def train_step(self, data):
        with tf.GradientTape() as tape:
            ap_distance, an_distance = self.siameseModel(data[0], training=True)
            loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, self.siameseModel.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siameseModel.trainable_weights))
        self.lossTracker.update_state(loss)
        return {'loss': self.lossTracker.result()}

    def test_step(self, data):
        ap_distance, an_distance = self.siameseModel(data[0], training=False)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        loss = tf.reduce_mean(loss)
        self.lossTracker.update_state(loss)
        return {'loss': self.lossTracker.result()}

    @property
    def metrics(self):
        return [self.lossTracker]
    




In [ ]:
siameseModel = SiameseModelClass(SiameseModel)
siameseModel.compile(optimizer=tf.keras.optimizers.Adam(1e-4))

# early stopping callback
cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-4,
    patience=5,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.5, 
                              patience=3, 
                              verbose=1, 
                              mode='min', 
                              min_lr=0.00001)

cb_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/siameseModel.keras',
    monitor= 'val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)

history = siameseModel.fit(batchGenerator(tripletsTrain, batchSize=128),
          steps_per_epoch=100,
          validation_data=batchGenerator(tripletsTest, batchSize=128),
          callbacks=[cb_early_stopping, cb_model_checkpoint,reduce_lr],
          validation_steps=100,
          epochs=128)


In [ ]:
# load best model 

def extractEncoder(model):
    encoder = getEmbeddingModel((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = extractEncoder(siameseModel)
encoder.save_weights("encoder.weights.h5")
encoder.summary()

In [ ]:
def classify_images(face_list1, face_list2, threshold=1.1):
    # Getting the encodings for the passed faces
    tensor1 = encoder.predict(face_list1)
    tensor2 = encoder.predict(face_list2)
    
    # Euclidean distance between the encodings
    distance = np.sqrt(np.sum(np.square(tensor1-tensor2), axis=-1))
    prediction = np.where(distance<=threshold, 0, 1)
    return prediction

In [ ]:
pos_list = np.array([])
neg_list = np.array([])

for data in batchGenerator(tripletsTest, batchSize=256):
    print(data)
    a, p, n = data[0]
    pos_list = np.append(pos_list, classify_images(a, p))
    neg_list = np.append(neg_list, classify_images(a, n))
    break

In [ ]:

def ModelMetrics(pos_list, neg_list):
    true = np.array([0]*len(pos_list)+[1]*len(neg_list))
    pred = np.append(pos_list, neg_list)
    
    # Compute and print the accuracy
    print(f"\nAccuracy of model: {accuracy_score(true, pred)}\n")
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(true, pred)

    categories  = ['Similar','Different']
    names = ['True Similar','False Similar', 'False Different','True Different']
    percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

    # Accuracy, Precision, Recall, F1-Score
    print("\nClassification Report:\n")
    print(classification_report(true, pred, target_names=categories))
ModelMetrics(pos_list, neg_list)

In [ ]:
# test for 2 images
face_1 = readImage(('bin', '0.jpg'))
face_2 = readImage(('bin', '1.jpg'))


face_1 = cv2.resize(face_1, (128, 128))
face_2 = cv2.resize(face_2, (128, 128))

# show images
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(face_1)
plt.title('Face 1')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(face_2)
plt.title('Face 2')
plt.axis('off')
plt.show()

face_1 = preprocess_input(face_1)
face_2 = preprocess_input(face_2)

#
face_1 = tf.convert_to_tensor(np.array([face_1], dtype="float32"))
face_2 = tf.convert_to_tensor(np.array([face_2], dtype="float32"))

prediction = classify_images(face_1, face_2, threshold=1.0)
print('Same person' if prediction[0]==0 else 'Different person')